In [ ]:
# %pip install python-gnupg
# %pip install azure-storage-file-datalake
# %restart_python

In [ ]:
import gnupg
import subprocess
from azure.storage.filedatalake import DataLakeServiceClient
from io import BytesIO
from datetime import datetime

In [ ]:
def get_datalake_client(account_name, account_key):
    service_client = DataLakeServiceClient(
        account_url=f"https://{account_name}.dfs.core.windows.net", credential=account_key
    )
    return service_client

service_client = get_datalake_client("qambusa", "F6Cxa90BGhRifHiFcxntdjM0yPm/Wsj3AFZedZJrpMimkHx/27ddUYwAdY5yZGv8XpYhzzjXQm7m+AStN8GtJA==")

def download_file_from_datalake(service_client, filesystem_name, directory_name, file_name):
    filesystem_client = service_client.get_file_system_client(filesystem_name)
    directory_client = filesystem_client.get_directory_client(directory_name)
    file_client = directory_client.get_file_client(file_name)
    
    download = file_client.download_file()
    file_data = download.readall()
    print(f"Downloaded {file_name} from Azure Data Lake")
    return file_data

In [ ]:
private_key_data = download_file_from_datalake(service_client, "clicmotion", "test/encrypted_keys", "private_key.asc")
#print(private_key_data)

In [ ]:
encrypted_file_data = download_file_from_datalake(service_client, "clicmotion", "test", "encrypted_data.txt.gpg")

In [ ]:
def gpg_decrypt_data(enc_data, passphrase):
    if isinstance(enc_data, str):
        enc_data = enc_data.encode()

    cmd = [
        "gpg", "--decrypt", "--batch", "--yes", "--no-tty",
        "--pinentry-mode", "loopback",
        "--passphrase", passphrase
    ]

    process = subprocess.Popen(cmd, 
                               stdin=subprocess.PIPE,
                               stdout=subprocess.PIPE, 
                               stderr=subprocess.PIPE,
                               )

    plaintext, err = process.communicate(input=enc_data)

    if process.returncode != 0:
        raise Exception(f"Decryption failed: {err.decode()}")

    return plaintext


In [ ]:
# Paste your actual private key content below, including all lines
def import_gpg_private_key(private_key_data):
    process = subprocess.Popen(
        [
            "gpg",
            "--batch",
            "--yes",
            "--import"
        ],
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE
    )
    stdout, stderr = process.communicate(input=private_key_data)
    if process.returncode != 0:
        raise Exception(f"Key import failed: {stderr.decode()}")

# Usage:
import_gpg_private_key(private_key_data)

In [ ]:
def upload_encrypted_data_to_datalake(service_client, filesystem_name, directory_name, file_name, encrypted_data):
    filesystem_client = service_client.get_file_system_client(filesystem_name)
    directory_client = filesystem_client.get_directory_client(directory_name)
    file_client = directory_client.get_file_client(file_name)
    
    file_client.upload_data(encrypted_data, overwrite=True)
    print(f"Encrypted file uploaded as {file_name}")

In [ ]:
upload_encrypted_data_to_datalake(service_client, "clicmotion", "test", "chetan_decrypted_data.txt", decrypted_data)